In [1]:
from plotnine import *
import numpy as np
import pandas as pd
import scipy as sp

# Plots for ordered multinomials

In [2]:
supp = np.linspace(-3,3)
n_ordered = 5

def plot_om(probs_fn, supp, K = 6):
    dfs = []
    for k in range(K):
        temp_df = pd.DataFrame(probs_fn(supp))
        temp_df['x'] = supp
        temp_df['run'] = f'Run {k+1}'
        dfs.append(temp_df)
    df = pd.concat(dfs)
        
    df = df.melt(id_vars=['x', 'run'])
    return (ggplot(df, aes(x='x', y='value', fill='variable')) + 
            geom_area() + 
            geom_vline(xintercept=0) + 
            guides(fill=False) +
            scale_x_continuous(breaks=range(int(min(supp)), int(max(supp))+1)) +
            theme(axis_title_x=element_blank(), axis_title_y=element_blank()) +
            facet_wrap('~run'))


In [ ]:
# Binomial

def binom_probs(supp):
    N = n_ordered -1
    cn = np.arange(N+1)
    p = sp.special.expit(supp)
    probs = sp.special.binom(N,cn) * \
        (p[:,None]**cn[None,:]) * \
        (1-p[:,None])**(N-cn[None,:])
    return probs
plot_om(binom_probs,supp,K=1)

In [4]:
# Cutpoints

def gen_cps(n_ordered):
    cp_scale = 1.5
    cp_len = n_ordered-1
    cps = np.sort(cp_scale*np.random.normal(size=cp_len))
    cps -= cps.mean()
    return cps

In [ ]:
# Probit

def probit_probs(supp):
    cps = gen_cps(n_ordered)

    ep = 0.5*(1+sp.special.erf( supp[:,None] - cps[None,:]))
    cprobs = np.concatenate([ np.ones( (supp.shape[0],1) ), ep, np.zeros( (supp.shape[0],1) )],axis=1)
    probs = cprobs[:,:-1]-cprobs[:,1:]
    return probs
plot_om(probit_probs,supp,K=6)

In [ ]:
# Logistic
def logistic_probs(supp):
    cps = gen_cps(n_ordered)
    pa = sp.special.expit(cps[None,:] - supp[:,None])
    probs = np.concatenate([ pa[:, 0, None], pa[:, 1:] - pa[:, :-1], 1.0 - pa[:, -1, None] ], axis=-1)
    return probs
plot_om(logistic_probs,supp,K=6)

In [ ]:
# Stereotype regression

def stereotype_probs(supp):
    phi_delta = np.random.dirichlet(np.ones(n_ordered-1))
    phi = np.concatenate([[0], np.cumsum(phi_delta)])
    s_mu = np.random.normal(size=n_ordered)
    probs = sp.special.softmax(s_mu[None,:] + phi[None,:]*supp[:,None], axis=-1)
    return probs
plot_om(stereotype_probs,supp,K=6)

# Real examples from regression
Averages of trust2_EU

In [ ]:
def real_logit(supp):
    cps = np.array([-0.83095682, -0.22333643, -0.09501735,  1.1493106 ])
    pa = sp.special.expit(cps[None,:] - supp[:,None])
    probs = np.concatenate([ pa[:, 0, None], pa[:, 1:] - pa[:, :-1], 1.0 - pa[:, -1, None] ], axis=-1)
    return probs

plot_om(real_logit,supp,K=1)

In [ ]:
# Values on some real data

def real_stereotype(supp):
    phi_delta = np.array([0.55368214, 0.19831634, 0.13467074, 0.11333078])
    phi = np.concatenate([[0], np.cumsum(phi_delta)])
    s_mu = np.array([ 0.59625734, -0.20218961, -1.68321933,  0.57362777,  0.71552383])
    probs = sp.special.softmax(s_mu[None,:] + phi[None,:]*supp[:,None], axis=-1)
    return probs
plot_om(real_stereotype,supp,K=1)

# Slide demo

In [ ]:
# Probit demo for slide

def probit_demo(supp):
    cps = np.array([-2.5,-0.8,-0.5,0.8])

    ep = sp.stats.norm.cdf(supp[:,None] - cps[None,:])
    cprobs = np.concatenate([ np.ones( (supp.shape[0],1) ), ep, np.zeros( (supp.shape[0],1) )],axis=1)
    probs = cprobs[:,:-1]-cprobs[:,1:]
    return probs
plot_om(probit_demo,np.linspace(-3,3),K=1)

In [ ]:
from scipy.stats import norm

# Generate standard normal curve data
x = np.linspace(-3.5, 3.5, 1000)
y = norm.pdf(x-1)
df = pd.DataFrame({"x": x, "y": y})

# Define vertical lines (z-scores) and their bands
bands = [-10,-2.5, -0.8, -0.5, 0.8, 10]
df["fill_id"] = pd.cut(
    df["x"], 
    bins=bands, 
    #labels=range(len(bands)-1), 
    include_lowest=True
)#.astype('str')

df

# Create the plot
(
    ggplot(df.iloc[::-1], aes(x="x", y="y"))
    + geom_area(aes(fill="fill_id"), show_legend=False)  # Color bands
    + scale_fill_discrete()
    + geom_vline(xintercept=bands[1:-1], color="black", size=0.5)  # Vertical lines
    + xlim(-3,3)
    + theme(
        axis_line=element_blank(),
        axis_title=element_blank(),
        #axis_text=element_blank(),
        #axis_ticks=element_blank(),
        
        figure_size=(6, 4)
    )
)